In [1]:
#!/usr/bin/env python3

import cv2
import depthai as dai
import numpy as np

# Closer-in minimum depth, disparity range is doubled (from 95 to 190):
extended_disparity = False
# Better accuracy for longer distance, fractional disparity 32-levels:
subpixel = False
# Better handling for occlusions:
lr_check = True

# Create pipeline
pipeline = dai.Pipeline()

# Define sources and outputs
monoLeft = pipeline.create(dai.node.MonoCamera)
monoRight = pipeline.create(dai.node.MonoCamera)
depth = pipeline.create(dai.node.StereoDepth)
xout = pipeline.create(dai.node.XLinkOut)

xout.setStreamName("disparity")

# Properties
monoLeft.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
monoLeft.setBoardSocket(dai.CameraBoardSocket.LEFT)
monoRight.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
monoRight.setBoardSocket(dai.CameraBoardSocket.RIGHT)

# Create a node that will produce the depth map (using disparity output as it's easier to visualize depth this way)
depth.setDefaultProfilePreset(dai.node.StereoDepth.PresetMode.HIGH_DENSITY)
# Options: MEDIAN_OFF, KERNEL_3x3, KERNEL_5x5, KERNEL_7x7 (default)
depth.initialConfig.setMedianFilter(dai.MedianFilter.KERNEL_7x7)
depth.setLeftRightCheck(lr_check)
depth.setExtendedDisparity(extended_disparity)
depth.setSubpixel(subpixel)

# Linking
monoLeft.out.link(depth.left)
monoRight.out.link(depth.right)
depth.disparity.link(xout.input)

# Connect to device and start pipeline
with dai.Device(pipeline) as device:

    # Output queue will be used to get the disparity frames from the outputs defined above
    q = device.getOutputQueue(name="disparity", maxSize=4, blocking=False)

    while True:
        inDisparity = q.get()  # blocking call, will wait until a new data has arrived
        frame = inDisparity.getFrame()
        # Normalization for better visualization
        frame = (frame * (255 / depth.initialConfig.getMaxDisparity())).astype(np.uint8)
        ##
        ##
        ##
        ##
        # Convert to graycsale
        #img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Blur the image for better edge detection
        img_blur = cv2.GaussianBlur(frame, (3,3), 0) 
        
        # Sobel Edge Detection
        sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
        sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
        sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection
        # Display Sobel Edge Detection Images
        # cv2.imshow('Sobel X', sobelx)
        # # cv2.waitKey(0)
        # cv2.imshow('Sobel Y', sobely)
        # # cv2.waitKey(0)
        cv2.imshow('Sobel X Y using Sobel() function', sobelxy)
        # cv2.waitKey(0)
        
        # Canny Edge Detection
        edges = cv2.Canny(image=img_blur, threshold1=30, threshold2=200) # Canny Edge Detection
        # Display Canny Edge Detection Image
        cv2.imshow('Canny Edge Detection', edges)
        ##
        ##
        ##
        ##
        # cv2.imshow("disparity", frame)

        # Available color maps: https://docs.opencv.org/3.4/d3/d50/group__imgproc__colormap.html
        frame = cv2.applyColorMap(frame, cv2.COLORMAP_JET)
        cv2.imshow("disparity_color", frame)

        if cv2.waitKey(1) == ord('q'):
            break

Switch between sobel filter kernels using keys '1' and '2'
Switching sobel filter kernel.
Switching sobel filter kernel.
Switching sobel filter kernel.
Switching sobel filter kernel.
Switching sobel filter kernel.


In [5]:
#!/usr/bin/env python3

import cv2
import numpy as np
import depthai as dai

# Weights to use when blending rgb/depth/confidence image
rgbWeight = 0.3
depthWeight = 0.3
confWeight = 0.3
# Normalized weights to use when blending rgb/depth/confidence image (should equal 1.0)
rgbWeightNorm = 0.3
depthWeightNorm = 0.3
confWeightNorm = 0.3


def updateRgbBlendWeights(percent):
    """
    Update the rgb weight used to blend rgb/depth/confidence image

    @param[in] percent The rgb weight expressed as a percentage (0..100)
    """
    global rgbWeight
    rgbWeight = float(percent)/100.0

def updateDepthBlendWeights(percent):
    """
    Update the depth weight used to blend rgb/depth/confidence image

    @param[in] percent The depth weight expressed as a percentage (0..100)
    """
    global depthWeight
    depthWeight = float(percent)/100.0

def updateConfBlendWeights(percent):
    """
    Update the confidence weight used to blend rgb/depth/confidence image

    @param[in] percent The confidence weight expressed as a percentage (0..100)
    """
    global confWeight
    confWeight = float(percent)/100.0

# Optional. If set (True), the ColorCamera is downscaled from 1080p to 720p.
# Otherwise (False), the aligned depth is automatically upscaled to 1080p
downscaleColor = True
fps = 30
# The disparity is computed at this resolution, then upscaled to RGB resolution
monoResolution = dai.MonoCameraProperties.SensorResolution.THE_720_P

# Create pipeline
pipeline = dai.Pipeline()
device = dai.Device()
queueNames = []

# Define sources and outputs
camRgb = pipeline.create(dai.node.ColorCamera)
left = pipeline.create(dai.node.MonoCamera)
right = pipeline.create(dai.node.MonoCamera)
stereo = pipeline.create(dai.node.StereoDepth)

rgbOut = pipeline.create(dai.node.XLinkOut)
disparityOut = pipeline.create(dai.node.XLinkOut)

rgbOut.setStreamName("rgb")
queueNames.append("rgb")
disparityOut.setStreamName("disp")
queueNames.append("disp")

#Properties
camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
camRgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
camRgb.setFps(fps)
if downscaleColor: camRgb.setIspScale(2, 3)
# For now, RGB needs fixed focus to properly align with depth.
# This value was used during calibration
try:
    calibData = device.readCalibration2()
    lensPosition = calibData.getLensPosition(dai.CameraBoardSocket.RGB)
    if lensPosition:
        camRgb.initialControl.setManualFocus(lensPosition)
except:
    raise

left.setResolution(monoResolution)
left.setBoardSocket(dai.CameraBoardSocket.LEFT)
left.setFps(fps)
right.setResolution(monoResolution)
right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
right.setFps(fps)

stereo.setDefaultProfilePreset(dai.node.StereoDepth.PresetMode.HIGH_DENSITY)
# LR-check is required for depth alignment
stereo.setLeftRightCheck(True)
if 0: stereo.setSubpixel(True)  # TODO enable for test
stereo.setDepthAlign(dai.CameraBoardSocket.RGB)

xoutConfMap = pipeline.create(dai.node.XLinkOut)
xoutConfMap.setStreamName('confidence_map')

# Linking
camRgb.isp.link(rgbOut.input)
left.out.link(stereo.left)
right.out.link(stereo.right)
stereo.disparity.link(disparityOut.input)
stereo.confidenceMap.link(xoutConfMap.input)

# Connect to device and start pipeline
with device:
    device.startPipeline(pipeline)

    frameRgb = None
    frameDisp = None
    frameC = None

    # Configure windows; trackbar adjusts blending ratio of rgb/depth
    rgbWindowName = "rgb"
    depthWindowName = "depth"
    confWindowName = "conf"
    blendedWindowName = "rgb-depth-conf"
    cv2.namedWindow(rgbWindowName)
    cv2.namedWindow(depthWindowName)
    cv2.namedWindow(confWindowName)
    cv2.namedWindow(blendedWindowName)
    cv2.createTrackbar('RGB Weight %', blendedWindowName, int(rgbWeight*100), 100, updateRgbBlendWeights)
    cv2.createTrackbar('Depth Weight %', blendedWindowName, int(depthWeight*100), 100, updateDepthBlendWeights)
    cv2.createTrackbar('Confidence Weight %', blendedWindowName, int(confWeight*100), 100, updateConfBlendWeights)
    while True:
        latestPacket = {}
        latestPacket["rgb"] = None
        latestPacket["disp"] = None
        latestPacket["confidence_map"] = None

        queueEvents = device.getQueueEvents(("rgb", "disp", "confidence_map"))
        for queueName in queueEvents:
            packets = device.getOutputQueue(queueName).tryGetAll()
            if len(packets) > 0:
                latestPacket[queueName] = packets[-1]

        if latestPacket["rgb"] is not None:
            frameRgb = latestPacket["rgb"].getCvFrame()
            cv2.imshow(rgbWindowName, frameRgb)

        if latestPacket["confidence_map"] is not None:
            frameC = latestPacket["confidence_map"].getCvFrame()
            cv2.imshow(confWindowName, frameC)

        if latestPacket["disp"] is not None:
            frameDisp = latestPacket["disp"].getFrame()
            maxDisparity = stereo.initialConfig.getMaxDisparity()
            # Optional, extend range 0..95 -> 0..255, for a better visualisation
            if 1: frameDisp = (frameDisp * 255. / maxDisparity).astype(np.uint8)
            # Optional, apply false colorization
            if 1: frameDisp = cv2.applyColorMap(frameDisp, cv2.COLORMAP_HOT)
            frameDisp = np.ascontiguousarray(frameDisp)
            cv2.imshow(depthWindowName, frameDisp)

        # Blend when all three frames received
        if frameRgb is not None and frameDisp is not None and frameC is not None:
            # Need to have all three frames in BGR format before blending
            if len(frameDisp.shape) < 3:
                frameDisp = cv2.cvtColor(frameDisp, cv2.COLOR_GRAY2BGR)
            if len(frameC.shape) < 3:
                frameC = cv2.cvtColor(frameC, cv2.COLOR_GRAY2BGR)
            sumWeight = rgbWeight + depthWeight + confWeight
            # Normalize the weights so their sum to be <= 1.0
            if sumWeight <= 1.0:
                rgbWeightNorm = rgbWeight
                depthWeightNorm = depthWeight
                confWeightNorm = confWeight
            else :
                rgbWeightNorm = rgbWeight / sumWeight
                depthWeightNorm = depthWeight / sumWeight
                confWeightNorm = confWeight / sumWeight
            blended1 = cv2.addWeighted(frameRgb, rgbWeightNorm, frameDisp, depthWeightNorm, 0)
            blended2 = cv2.addWeighted(blended1, rgbWeightNorm + depthWeightNorm, frameC, confWeightNorm, 0)
            cv2.imshow(blendedWindowName, blended2)
            frameRgb = None
            frameDisp = None
            frameC = None

        if cv2.waitKey(1) == ord('q'):
            break
        

KeyboardInterrupt: 

<class 'depthai.Device'>


In [4]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 tensorflow-hub opencv-python matplotlib
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import time
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']
# Id	Part
# 0	nose
# 1	leftEye
# 2	rightEye
# 3	leftEar
# 4	rightEar
# 5	leftShoulder
# 6	rightShoulder
# 7	leftElbow
# 8	rightElbow
# # 9	leftWrist
# # 10	rightWrist
# # 11	leftHip
# # 12	rightHip
# # 13	leftKnee
# # 14	rightKnee
# # 15	leftAnkle
# # 16	rightAnkle



## Implementation of Sobel Filter on Depth camera output - which will be further processed with MoveNet pose estimation (probably??)
import depthai as dai

# Create pipeline
pipeline = dai.Pipeline()

# # Define sources and outputs
# camRgb = pipeline.create(dai.node.ColorCamera)
# monoLeft = pipeline.create(dai.node.MonoCamera)
# monoRight = pipeline.create(dai.node.MonoCamera)

# edgeDetectorLeft = pipeline.create(dai.node.EdgeDetector)
# edgeDetectorRight = pipeline.create(dai.node.EdgeDetector)
# edgeDetectorRgb = pipeline.create(dai.node.EdgeDetector)

# xoutEdgeLeft = pipeline.create(dai.node.XLinkOut)
# xoutEdgeRight = pipeline.create(dai.node.XLinkOut)
# xoutEdgeRgb = pipeline.create(dai.node.XLinkOut)
# xinEdgeCfg = pipeline.create(dai.node.XLinkIn)

# edgeLeftStr = "edge left"
# edgeRightStr = "edge right"
# edgeRgbStr = "edge rgb"
# edgeCfgStr = "edge cfg"

# xoutEdgeLeft.setStreamName(edgeLeftStr)
# xoutEdgeRight.setStreamName(edgeRightStr)
# xoutEdgeRgb.setStreamName(edgeRgbStr)
# xinEdgeCfg.setStreamName(edgeCfgStr)

# # Properties
# camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
# camRgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

# monoLeft.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
# monoLeft.setBoardSocket(dai.CameraBoardSocket.LEFT)
# monoRight.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
# monoRight.setBoardSocket(dai.CameraBoardSocket.RIGHT)

# edgeDetectorRgb.setMaxOutputFrameSize(camRgb.getVideoWidth() * camRgb.getVideoHeight())

# # Linking
# monoLeft.out.link(edgeDetectorLeft.inputImage)
# monoRight.out.link(edgeDetectorRight.inputImage)
# camRgb.video.link(edgeDetectorRgb.inputImage)

# edgeDetectorLeft.outputImage.link(xoutEdgeLeft.input)
# edgeDetectorRight.outputImage.link(xoutEdgeRight.input)
# edgeDetectorRgb.outputImage.link(xoutEdgeRgb.input)

# xinEdgeCfg.out.link(edgeDetectorLeft.inputConfig)
# xinEdgeCfg.out.link(edgeDetectorRight.inputConfig)
# xinEdgeCfg.out.link(edgeDetectorRgb.inputConfig)


# # ####
# #####
# #####
# ####
# ###
# #####
# # Define source and output
# camRgb = pipeline.create(dai.node.ColorCamera)
# xoutRgb = pipeline.create(dai.node.XLinkOut)

# xoutRgb.setStreamName("rgb")

# # Properties
# camRgb.setPreviewSize(300, 300)
# camRgb.setInterleaved(False)
# camRgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.RGB)

# # Linking
# camRgb.preview.link(xoutRgb.input)
# #####
##
##
##
##

## Depth Stream 
##
#
#

# Closer-in minimum depth, disparity range is doubled (from 95 to 190):
extended_disparity = False
# Better accuracy for longer distance, fractional disparity 32-levels:
subpixel = False
# Better handling for occlusions:
lr_check = True

# Create pipeline
# pipeline = dai.Pipeline()

# Define sources and outputs
monoLeft = pipeline.create(dai.node.MonoCamera)
monoRight = pipeline.create(dai.node.MonoCamera)
depth = pipeline.create(dai.node.StereoDepth)
xout = pipeline.create(dai.node.XLinkOut)

xout.setStreamName("disparity")

# Properties
monoLeft.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
monoLeft.setBoardSocket(dai.CameraBoardSocket.LEFT)
monoRight.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
monoRight.setBoardSocket(dai.CameraBoardSocket.RIGHT)

# Create a node that will produce the depth map (using disparity output as it's easier to visualize depth this way)
depth.setDefaultProfilePreset(dai.node.StereoDepth.PresetMode.HIGH_DENSITY)
# Options: MEDIAN_OFF, KERNEL_3x3, KERNEL_5x5, KERNEL_7x7 (default)
depth.initialConfig.setMedianFilter(dai.MedianFilter.KERNEL_7x7)
depth.setLeftRightCheck(lr_check)
depth.setExtendedDisparity(extended_disparity)
depth.setSubpixel(subpixel)

# Linking
monoLeft.out.link(depth.left)
monoRight.out.link(depth.right)
depth.disparity.link(xout.input)


ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==2.4.1

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Function to loop through each person detected and render
## Cap 1 (MAIN CAMERA/FRONT FACING) Mesh generation logic cap1 = cv2.VideoCapture(2)
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    i = 0;
    for person in keypoints_with_scores:
                        #CURRENTLY WE ARE DETECTING AND BUILDING MESH LISTS FOR ONLY 2 PEOPLE
                        #IF WE HAVE TO BUILD MESHES FOR MORE THAN 2 PEOPLE, WE NEED TO ADD LIST3 LIST4...LIST6 in the code below
                        
        if i ==0:       #THINK OF A BETTER LOGIC HERE THAT LEVERAGES KEYPOINTS_SCORES.SHAPE
            list1 =[]   
            list1 = draw_connections(frame, person, edges, confidence_threshold, list1)
            # print(list1)
            draw_keypoints(frame, person, confidence_threshold)
        if i ==1:   
            list2=[]
            list2 = draw_connections(frame, person, edges, confidence_threshold, list2)
            # print(list2)
            draw_keypoints(frame, person, confidence_threshold)

        i = i+1
    return list1, list2
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)
EDGES = {
    (0, 4): 'c',            #self added
    (0, 3): 'm',            #self added
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (5, 12): 'm',           # self added
    (6, 11): 'm',           # self added
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}
def draw_connections(frame, keypoints, edges, confidence_threshold, list):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        #print(p1,p2)
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)
            ##
            ##Check if the p1 and p2 points are face or torso or limbs landmarks 
            # and draw elipse or circle or rectangel accordingly to estimate mesh 
            ##
            if p1 ==0 and p2 ==4:
                    radius_of_head = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )))
                    cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_head, color=(255,0,0),thickness=-1)
                    list_temp = [int(x1),int(y1), int(radius_of_head), 'HEAD_0', 1]
                    list.append(list_temp)
                    

            if p1 ==0 and p2 ==3:
                    radius_of_head = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )))
                    cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_head, color=(255,0,0),thickness=-1)
                    list_temp = [int(x1),int(y1), int(radius_of_head), 'HEAD_1', 1]
                    list.append(list_temp)

            if p1 ==0 and p2 ==1:
                continue
            
            if p1 ==0 and p2 ==2:
                continue
            
            if p1 ==1 and p2 ==3:
                radius_of_head = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )))
                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_head, color=(255,0,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_head), 'HEAD_2', 1]
                list.append(list_temp)
            
            if p1 ==2 and p2 ==4:
                radius_of_head = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )))
                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_head, color=(255,0,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_head), 'HEAD_2', 1]
                list.append(list_temp)
                
            if p1 ==0 and p2 ==5:
                
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'LEFT_SHOULDER_BLADE_1', 1]
                list.append(list_temp)

                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'LEFT_SHOULDER_BLADE_2', 1]
                list.append(list_temp)
                #Not much logic to drawing any bounding box in this region. Potentiall a Triangle can be drawn here. Will research that to further imporve mesh quality
            
            if p1 ==0 and p2 ==6:
                
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'RIGHT_SHOULDER_BLADE_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'RIGHT_SHOULDER_BLADE_2', 1]
                list.append(list_temp)
                #Not much logic to drawing any bounding box in this region. Potentiall a Triangle can be drawn here. Will research that to further imporve mesh quality

            if p1 ==5 and p2 ==7:
               ##
               ## Drawing Elipses mesh
               ##
                # BLUE = (255, 0, 0)
                # center = round((int(x1)+int(x2))/2), round((int(y1)+int(y2))/2)
                # distance_between_landmarks = math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )
                # axes = int(round(distance_between_landmarks/2)) , int(round(distance_between_landmarks/4))
                # if x1 > x2 and y1 > y2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # if x1 > x2 and y1 < y2:
                #     angle = 360 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 > y2:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 < y2:
                #     angle = -180 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                ##
                ## Drawing Circle mesh
                ##
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'LEFT_UPPERARM_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'LEFT_UPPERARM_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'LEFT_UPPERARM_3', 1]
                list.append(list_temp)

                
            if p1 ==7 and p2 ==9:
                # BLUE = (255, 0, 0)
                # center = round((int(x1)+int(x2))/2), round((int(y1)+int(y2))/2)
                # distance_between_landmarks = math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )
                # axes = int(round(distance_between_landmarks/2)) , int(round(distance_between_landmarks/4))
                # if x1 > x2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # else:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)

                # if x1 > x2 and y1 > y2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # if x1 > x2 and y1 < y2:
                #     angle = 360 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 > y2:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 < y2:
                #     angle = -180 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                ##
                ## Drawing Circle mesh
                ##
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'LEFT_LOWERARM_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=round(radius_of_mesh_circle*1.5), color=(0,255,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'LEFT_LOWERARM_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'LEFT_LOWERARM_3', 1]
                list.append(list_temp)


            if p1 ==6 and p2 ==8:
                # BLUE = (255, 0, 0)
                # center = round((int(x1)+int(x2))/2), round((int(y1)+int(y2))/2)
                # distance_between_landmarks = math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )
                # axes = int(round(distance_between_landmarks/2)) , int(round(distance_between_landmarks/4))
                # if x1 > x2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # else:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)

                # if x1 > x2 and y1 > y2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # if x1 > x2 and y1 < y2:
                #     angle = 360 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 > y2:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 < y2:
                #     angle = -180 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
           
                ##
                ## Drawing Circle mesh
                ##
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'RIGHT_UPPERARM_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'RIGHT_UPPERARM_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'RIGHT_UPPERARM_3', 1]
                list.append(list_temp)

            if p1 ==8 and p2 ==10:
                # BLUE = (255, 0, 0)
                # center = round((int(x1)+int(x2))/2), round((int(y1)+int(y2))/2)
                # distance_between_landmarks = math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )
                # axes = int(round(distance_between_landmarks/2)) , int(round(distance_between_landmarks/4))
                # if x1 > x2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # else:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)

                # if x1 > x2 and y1 > y2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # if x1 > x2 and y1 < y2:
                #     angle = 360 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 > y2:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 < y2:
                #     angle = -180 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                ##
                ## Drawing Circle mesh
                ##
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'RIGHT_LOWERARM_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius= round(radius_of_mesh_circle*1.5), color=(0,255,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'RIGHT_LOWERARM_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'RIGHT_LOWERARM_3', 1]
                list.append(list_temp)

            if p1 ==5 and p2 ==6:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/2))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'COLLAR_BONE', 1]
                list.append(list_temp)
            #Torso
            
            if p1 ==5 and p2 ==11:
                continue
            #Torso
            
            if p1 ==6 and p2 ==12:
                continue
            #Torso
            
            if p1 ==11 and p2 ==12:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/2))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'HIP_BONE', 1]
                list.append(list_temp)

            #Torso 
            
            if p1 ==5 and p2 ==12:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'TORSO_1', 1]
                list.append(list_temp)
                # frame = cv2.rectangle(frame, start_point = (int(x1),int(y1)), end_point = (int(x2),int(y2)), color = (255, 0, 0), thickness= -1)
                # cv2.imshow("Torso detected",frame)
                # continue
            #Torso
            
            if p1 ==6 and p2==11:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'TORSO_2', 1]
                list.append(list_temp)

                # continue
                # frame = cv2.rectangle(frame, start_point = (int(x1),int(y1)), end_point = (int(x2),int(y2)), color = (255, 0, 0), thickness= -1)
                # cv2.imshow("Torso detected",frame)
                # #cv2.rectangle(frame, start_point = (x1,y1), end_point = (x2,y2), color = (255, 0, 0), thickness= -1)
            #Torso
 
            if p1 ==11 and p2 ==13:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'LEFT_UPPERLEG_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'LEFT_UPPERLEG_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'LEFT_UPPERLEG_3', 1]
                list.append(list_temp)

            #Legs

            if p1 ==13 and p2 ==15:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'LEFT_LOWERLEG_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=round(radius_of_mesh_circle*1.5), color=(0,125,125),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'LEFT_LOWERLEG_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'LEFT_LOWERLEG_3', 1]
                list.append(list_temp)
            # Legs
            if p1 ==12 and p2 ==14:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'RIGHT_UPPERLEG_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'RIGHT_UPPERLEG_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'RIGHT_UPPERLEG_3', 1]
                list.append(list_temp)

            #Legs
            if p1 ==14 and p2 ==16:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'RIGHT_LOWERLEG_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=round(radius_of_mesh_circle*1.5), color=(0,125,125),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'RIGHT_LOWERLEG_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'RIGHT_LOWERLEG_3', 1]
                list.append(list_temp)
            
            #Legs
    return list

##
## Cap 3  (SIDE ANGLE CAM) Mesh Generation Logic cap3 = cv2.VideoCapture(0)
##
def loop_through_people_3(frame, keypoints_with_scores, edges, confidence_threshold):
    i = 0;
    for person in keypoints_with_scores:
                        #CURRENTLY WE ARE DETECTING AND BUILDING MESH LISTS FOR ONLY 2 PEOPLE
                        #IF WE HAVE TO BUILD MESHES FOR MORE THAN 2 PEOPLE, WE NEED TO ADD LIST3 LIST4...LIST6 in the code below
                        
        if i ==0:       #THINK OF A BETTER LOGIC HERE THAT LEVERAGES KEYPOINTS_SCORES.SHAPE
            list1 =[]   
            list1 = draw_connections_3(frame, person, edges, confidence_threshold, list1)
            # print(list1)
            draw_keypoints(frame, person, confidence_threshold)
        if i ==1:   
            list2=[]
            list2 = draw_connections_3(frame, person, edges, confidence_threshold, list2)
            # print(list2)
            draw_keypoints(frame, person, confidence_threshold)

        i = i+1
    return list1, list2
def draw_connections_3(frame, keypoints, edges, confidence_threshold, list):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        #print(p1,p2)
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)
            ##
            ##Check if the p1 and p2 points are face or torso or limbs landmarks 
            # and draw elipse or circle or rectangel accordingly to estimate mesh 
            ##
            if p1 ==0 and p2 ==4:
                    radius_of_head = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )))
                    cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_head, color=(255,0,0),thickness=-1)
                    list_temp = [int(x2),int(y2), int(radius_of_head), 'HEAD_0', 1]
                    list.append(list_temp)
                    

            if p1 ==0 and p2 ==3:
                    radius_of_head = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )))
                    cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_head, color=(255,0,0),thickness=-1)
                    list_temp = [int(x2),int(y2), int(radius_of_head), 'HEAD_1', 1]
                    list.append(list_temp)

            if p1 ==0 and p2 ==1:
                continue
            
            if p1 ==0 and p2 ==2:
                continue
            
            if p1 ==1 and p2 ==3:
                radius_of_head = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )))
                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_head, color=(255,0,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_head), 'HEAD_2', 1]
                list.append(list_temp)
            
            if p1 ==2 and p2 ==4:
                radius_of_head = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )))
                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_head, color=(255,0,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_head), 'HEAD_2', 1]
                list.append(list_temp)
                
            if p1 ==0 and p2 ==5:
                
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'LEFT_SHOULDER_BLADE_1', 1]
                list.append(list_temp)

                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'LEFT_SHOULDER_BLADE_2', 1]
                list.append(list_temp)
                #Not much logic to drawing any bounding box in this region. Potentiall a Triangle can be drawn here. Will research that to further imporve mesh quality
            
            if p1 ==0 and p2 ==6:
                
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'RIGHT_SHOULDER_BLADE_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'RIGHT_SHOULDER_BLADE_2', 1]
                list.append(list_temp)
                #Not much logic to drawing any bounding box in this region. Potentiall a Triangle can be drawn here. Will research that to further imporve mesh quality

            if p1 ==5 and p2 ==7:
               ##
               ## Drawing Elipses mesh
               ##
                # BLUE = (255, 0, 0)
                # center = round((int(x1)+int(x2))/2), round((int(y1)+int(y2))/2)
                # distance_between_landmarks = math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )
                # axes = int(round(distance_between_landmarks/2)) , int(round(distance_between_landmarks/4))
                # if x1 > x2 and y1 > y2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # if x1 > x2 and y1 < y2:
                #     angle = 360 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 > y2:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 < y2:
                #     angle = -180 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                ##
                ## Drawing Circle mesh
                ##
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'LEFT_UPPERARM_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'LEFT_UPPERARM_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'LEFT_UPPERARM_3', 1]
                list.append(list_temp)

                
            if p1 ==7 and p2 ==9:
                # BLUE = (255, 0, 0)
                # center = round((int(x1)+int(x2))/2), round((int(y1)+int(y2))/2)
                # distance_between_landmarks = math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )
                # axes = int(round(distance_between_landmarks/2)) , int(round(distance_between_landmarks/4))
                # if x1 > x2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # else:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)

                # if x1 > x2 and y1 > y2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # if x1 > x2 and y1 < y2:
                #     angle = 360 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 > y2:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 < y2:
                #     angle = -180 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                ##
                ## Drawing Circle mesh
                ##
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'LEFT_LOWERARM_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=round(radius_of_mesh_circle*1.5), color=(0,255,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'LEFT_LOWERARM_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'LEFT_LOWERARM_3', 1]
                list.append(list_temp)


            if p1 ==6 and p2 ==8:
                # BLUE = (255, 0, 0)
                # center = round((int(x1)+int(x2))/2), round((int(y1)+int(y2))/2)
                # distance_between_landmarks = math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )
                # axes = int(round(distance_between_landmarks/2)) , int(round(distance_between_landmarks/4))
                # if x1 > x2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # else:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)

                # if x1 > x2 and y1 > y2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # if x1 > x2 and y1 < y2:
                #     angle = 360 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 > y2:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 < y2:
                #     angle = -180 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
           
                ##
                ## Drawing Circle mesh
                ##
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'RIGHT_UPPERARM_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'RIGHT_UPPERARM_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'RIGHT_UPPERARM_3', 1]
                list.append(list_temp)

            if p1 ==8 and p2 ==10:
                # BLUE = (255, 0, 0)
                # center = round((int(x1)+int(x2))/2), round((int(y1)+int(y2))/2)
                # distance_between_landmarks = math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )
                # axes = int(round(distance_between_landmarks/2)) , int(round(distance_between_landmarks/4))
                # if x1 > x2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # else:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)

                # if x1 > x2 and y1 > y2:
                #     angle = 180 - (int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi)))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                # if x1 > x2 and y1 < y2:
                #     angle = 360 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 > y2:
                #     angle = int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                
                # if x1 < x2 and y1 < y2:
                #     angle = -180 - int(round (math.acos((math.sqrt((int(x1)-int(x2))**2) / distance_between_landmarks)) * 180/math.pi))
                #     cv2.ellipse(frame, center, axes, angle, 0.0, 360.0, BLUE, -1)
                ##
                ## Drawing Circle mesh
                ##
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'RIGHT_LOWERARM_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=round(radius_of_mesh_circle*1.5), color=(0,255,0),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'RIGHT_LOWERARM_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,255,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'RIGHT_LOWERARM_3', 1]
                list.append(list_temp)

            if p1 ==5 and p2 ==6:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/2))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'COLLAR_BONE', 1]
                list.append(list_temp)
            #Torso
            
            if p1 ==5 and p2 ==11:
                continue
            #Torso
            
            if p1 ==6 and p2 ==12:
                continue
            #Torso
            
            if p1 ==11 and p2 ==12:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/2))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'HIP_BONE', 1]
                list.append(list_temp)

            #Torso 
            
            if p1 ==5 and p2 ==12:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'TORSO_1', 1]
                list.append(list_temp)
                # frame = cv2.rectangle(frame, start_point = (int(x1),int(y1)), end_point = (int(x2),int(y2)), color = (255, 0, 0), thickness= -1)
                # cv2.imshow("Torso detected",frame)
                # continue
            #Torso
            
            if p1 ==6 and p2==11:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(255,0,0),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'TORSO_2', 1]
                list.append(list_temp)

                # continue
                # frame = cv2.rectangle(frame, start_point = (int(x1),int(y1)), end_point = (int(x2),int(y2)), color = (255, 0, 0), thickness= -1)
                # cv2.imshow("Torso detected",frame)
                # #cv2.rectangle(frame, start_point = (x1,y1), end_point = (x2,y2), color = (255, 0, 0), thickness= -1)
            #Torso
 
            if p1 ==11 and p2 ==13:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'LEFT_UPPERLEG_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'LEFT_UPPERLEG_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'LEFT_UPPERLEG_3', 1]
                list.append(list_temp)

            #Legs

            if p1 ==13 and p2 ==15:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'LEFT_LOWERLEG_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=round(radius_of_mesh_circle*1.5), color=(0,125,125),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'LEFT_LOWERLEG_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'LEFT_LOWERLEG_3', 1]
                list.append(list_temp)
            # Legs
            if p1 ==12 and p2 ==14:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'RIGHT_UPPERLEG_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'RIGHT_UPPERLEG_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'RIGHT_UPPERLEG_3', 1]
                list.append(list_temp)

            #Legs
            if p1 ==14 and p2 ==16:
                radius_of_mesh_circle = round(int(math.sqrt( (int(x1)-int(x2))**2 + (int(y1)-int(y2))**2 )/4))
                cp_x , cp_y= int(round((x1+x2)/2)) , int(round((y1+y2)/2))

                cv2.circle(frame, (int(x1),int(y1)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(x1),int(y1), int(radius_of_mesh_circle), 'RIGHT_LOWERLEG_1', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(x2),int(y2)), radius=round(radius_of_mesh_circle*1.5), color=(0,125,125),thickness=-1)
                list_temp = [int(x2),int(y2), int(radius_of_mesh_circle), 'RIGHT_LOWERLEG_2', 1]
                list.append(list_temp)
                cv2.circle(frame, (int(cp_x),int(cp_y)), radius=radius_of_mesh_circle, color=(0,125,125),thickness=-1)
                list_temp = [int(cp_x),int(cp_y), int(radius_of_mesh_circle), 'RIGHT_LOWERLEG_3', 1]
                list.append(list_temp)
            
            #Legs
    return list
#######################################################
#######################################################
#Make Detections
#######################################################
#######################################################
cap1 = cv2.VideoCapture(2)
cap2 = cv2.VideoCapture(1)
cap3 = cv2.VideoCapture(0)

# cap = cv2.VideoCaptu!cd DenseDepth; python test.pyre('Lumbar-Evalution-PalpationVideo.mp4')
# cap = cv2.VideoCapture('Back-Eval-Test1.mp4')
# cap = cv2.VideoCapture('novak.mp4')
#######################################################
#######################################################

with dai.Device(pipeline) as device:
    ##OAK D RGB Cameras
    # qRgb = device.getOutputQueue(name="rgb", maxSize=2, blocking=False)
    # qLeft = device.getOutputQueue(name="left", maxSize=2, blocking=False)
    # qRight = device.getOutputQueue(name="right", maxSize=2, blocking=False)

    # print('Connected cameras:', device.getConnectedCameras())
    # # Print out usb speed
    # print('Usb speed:', device.getUsbSpeed().name)
    # # Bootloader version
    # if device.getBootloaderVersion() is not None:
    #     print('Bootloader version:', device.getBootloaderVersion())
    # # Device name
    # print('Device name:', device.getDeviceName())
    # # Output queue will be used to get the rgb frames from the output defined above
    # qRgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

    ##OAK D Stereo vision
  
    #  # Output queue will be used to get the disparity frames from the outputs defined above
    q = device.getOutputQueue(name="disparity", maxSize=4, blocking=False)
    
    # ##Sobel Filter
    # edgeLeftQueue = device.getOutputQueue(edgeLeftStr, 8, False)
    # edgeRightQueue = device.getOutputQueue(edgeRightStr, 8, False)
    # edgeRgbQueue = device.getOutputQueue(edgeRgbStr, 8, False)
    # edgeCfgQueue = device.getInputQueue(edgeCfgStr)

    # print("Switch between sobel filter kernels using keys '1' and '2'")
    ##
    ##
    ##
    ##
    ##
    while cap1.isOpened() and cap2.isOpened() and cap3.isOpened():
        ##
        ##
        ##OAKD RGB cam processing:
        ##
        # ## 
        # inRgb = qRgb.get()  # blocking call, will wait until a new data has arrived
        # inLeft = qLeft.get()  # blocking call, will wait until a new data has arrived
        # inRight = qRight.get()  # blocking call, will wait until a new data has arr.ived
        # ##
        # inRgb = qRgb.get()  # blocking call, will wait until a new data has arrived

        ##
        ##OAKD Stereo Vision processing
        inDisparity = q.get()  # blocking call, will wait until a new data has arrived
        frame = inDisparity.getFrame()
        # Normalization for better visualization
        frame = (frame * (255 / depth.initialConfig.getMaxDisparity())).astype(np.uint8)
        ##
        # ## Edge Detection via Sobel Filter                
        # edgeLeft = edgeLeftQueue.get()
        # edgeRight = edgeRightQueue.get()
        # edgeRgb = edgeRgbQueue.get()

        # edgeLeftFrame = edgeLeft.getFrame()
        # edgeRightFrame = edgeRight.getFrame()
        # edgeRgbFrame = edgeRgb.getFrame()

        # gray = cv2.cvtColor(edgeRgbFrame,cv2.COLOR_BGR2GRAY)
        # gray = np.float32(gray)
        # dst = cv2.cornerHarris(edgeRgbFrame,2,3,0.04)
        # #result is dilated for marking the corners, not important
        # dst = cv2.dilate(dst,None)
        # # Threshold for an optimal value, it may vary depending on the image.
        # edgeRgbFrame[dst>0.01*dst.max()]=[0,0,255]
        # cv2.imshow('dst',edgeRgbFrame)

        ##
        ##Standard RGB cam processing
        ##
        ##
        # ret1, frame1 = cap1.read()
        # Resize image
        ret1, frame1 = cap1.read()
        img1 = frame1.copy()
        img1 = tf.image.resize_with_pad(tf.expand_dims(img1, axis=0), 192,256)
        input_img = tf.cast(img1, dtype=tf.int32)
        # Detection section
        results = movenet(input_img)
        keypoints_with_scores_1 = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        # Render keypoints 
        
        ## TESTING TO ADD CAM STREAM 2 here
        ret2, frame2 = cap2.read() # either we read the frame now in the IF condition; or we read it in each while loop iteration (seems like less processing power would be required in former approach)
        img2 = frame2.copy()
        img2 = tf.image.resize_with_pad(tf.expand_dims(img2, axis=0), 192,256)
        input_img_2 = tf.cast(img2, dtype=tf.int32)
        # Detection section
        results_2 = movenet(input_img_2)
        keypoints_with_scores_2 = results_2['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        #
        ##CAM 3
        ret3, frame3 = cap3.read()
        # Resize image
        img3 = frame3.copy()
        img3 = tf.image.resize_with_pad(tf.expand_dims(img3, axis=0), 192,256)
        input_img_3 = tf.cast(img3, dtype=tf.int32)
        # Detection section
        results_3 = movenet(input_img_3)
        keypoints_with_scores_3 = results_3['output_0'].numpy()[:,:,:51].reshape((6,17,3))

        # ##CAM 4
        # ret4, frame4 = inRgb.getCvFrame()
        # # Resize image
        # img4 = frame4.copy()
        # img4 = tf.image.resize_with_pad(tf.expand_dims(img4, axis=0), 192,256)
        # input_img_4 = tf.cast(img4, dtype=tf.int32)
        # # Detection section
        # results_4 = movenet(input_img_4)
        # keypoints_with_scores_4 = results_4['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        # ##
        # ##
        #clean all lists here
        list1, list2 = loop_through_people(frame1, keypoints_with_scores_1, EDGES, 0.3)
        list3, list4 = loop_through_people(frame2, keypoints_with_scores_2, EDGES, 0.3)
        list5, list6 = loop_through_people_3(frame3, keypoints_with_scores_3, EDGES, 0.3)
        # list7, list8 = loop_through_people(frame4, keypoints_with_scores_4, EDGES, 0.3)

        #
        # print(list1)
        # print(list2)
        ##
        ## Collision Detection 
        ## compare lists to find collision detection
        ##
        ##
        if len(list1) > 0 and len(list2) > 0:
            for circle_person_1 in list1:
                for circle_person_2 in list2:
                    # print(circle_person_1)
                    # print(circle_person_2)
                    x1 = circle_person_1[0]
                    y1 = circle_person_1[1]
                    r1 = circle_person_1[2]
                    x2 = circle_person_2[0]
                    y2 = circle_person_2[1]
                    r2 = circle_person_2[2]
                    distance_between_centers = round(math.sqrt((int(x1)-int(x2))**2 + (int(y1)-int(y2))**2))
                    # print(distance_between_centers)
                    if distance_between_centers < (r1 + r2):  
                        ## Switching from stream 1 to stream 2 
                        ## logic should be implemented here
                        # ret2, frame2 = cap2.read() # either we read the frame now in the IF condition; or we read it in each while loop iteration (seems like less processing power would be required in former approach)
                        # img2 = frame2.copy()
                        # img2 = tf.image.resize_with_pad(tf.expand_dims(img2, axis=0), 192,256)
                        # input_img = tf.cast(img2, dtype=tf.int32)
                        # # Detection section
                        # results = movenet(input_img)
                        # keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
                        # Render keypoints 
                            #clean all lists here
                        # list3, list4 = loop_through_people(frame2, keypoints_with_scores_2, EDGES, 0.3)
                        ##
                        #3
                        if len(list3) > 0 and len(list4) > 0:
                            for circle_person_3 in list3:
                                for circle_person_4 in list4:
                                    # print(circle_person_1)
                                    # print(circle_person_2)
                                    x3 = circle_person_3[0]
                                    y3 = circle_person_3[1]
                                    r3 = circle_person_3[2]
                                    x4 = circle_person_4[0]
                                    y4 = circle_person_4[1]
                                    r4 = circle_person_4[2]
                                    distance_between_centers_frame2 = round(math.sqrt((int(x3)-int(x4))**2 + (int(y3)-int(y4))**2))
                                    # print(distance_between_centers)
                                    if distance_between_centers_frame2 < (r3 + r4):  
                                        ##Draw red circle at the point of touch
                                        cv2.circle(frame1, (int(x1),int(y1)), radius=int(r1), color=(0,0,255),thickness=-1)
                                        cv2.circle(frame2, (int(x3),int(y3)), radius=int(r3), color=(0,0,255),thickness=-1)
                                        #Print out the point of touch (as per the MAIN CAMERA STREAM i.e  frame0)
                                        string = circle_person_1[3] + 'touched' + circle_person_2[3]
                                        print(string)

                        if len(list5) > 0 and len(list6) > 0:
                            for circle_person_5 in list5:
                                for circle_person_6 in list6:
                                    # print(circle_person_1)
                                    # print(circle_person_2)
                                    x5 = circle_person_5[0]
                                    y5 = circle_person_5[1]
                                    r5 = circle_person_5[2]
                                    x6 = circle_person_6[0]
                                    y6 = circle_person_6[1]
                                    r6 = circle_person_6[2]
                                    distance_between_centers_frame3 = round(math.sqrt((int(x5)-int(x6))**2 + (int(y5)-int(y6))**2))
                                    # print(distance_between_centers)
                                    if distance_between_centers_frame3 < (r5 + r6):  
                                        ##Draw red circle at the point of touch
                                        cv2.circle(frame1, (int(x1),int(y1)), radius=int(r1), color=(0,0,255),thickness=-1)
                                        cv2.circle(frame3, (int(x5),int(y5)), radius=int(r5), color=(0,0,255),thickness=-1)
                                        #Print out the point of touch (as per the MAIN CAMERA STREAM i.e  frame0)
                                        string = circle_person_1[3] + 'touched' + circle_person_2[3]
                                        print(string)

                        # if len(list7) > 0 and len(list8) > 0:
                        #     for circle_person_7 in list7:
                        #         for circle_person_8 in list8:
                        #             # print(circle_person_1)
                        #             # print(circle_person_2)
                        #             x7 = circle_person_7[0]
                        #             y7 = circle_person_7[1]
                        #             r7 = circle_person_7[2]
                        #             x8 = circle_person_8[0]
                        #             y8 = circle_person_8[1]
                        #             r8 = circle_person_8[2]
                        #             distance_between_centers_frame4 = round(math.sqrt((int(x7)-int(x8))**2 + (int(y7)-int(y8))**2))
                        #             # print(distance_between_centers)
                        #             if distance_between_centers_frame4 < (r7 + r8):  
                        #                 ##Draw red circle at the point of touch
                        #                 cv2.circle(frame1, (int(x1),int(y1)), radius=int(r1), color=(0,0,255),thickness=-1)
                        #                 cv2.circle(frame4, (int(x7),int(y7)), radius=int(r7), color=(0,0,255),thickness=-1)
                        #                 #Print out the point of touch (as per the MAIN CAMERA STREAM i.e  frame0)
                        #                 string = circle_person_1[3] + 'touched' + circle_person_2[3]
                        #                 print(string)
                        


        # Show the frame
        # cv2.imshow(edgeLeftStr, edgeLeftFrame)
        # cv2.imshow(edgeRightStr, edgeRightFrame)
        # cv2.imshow(edgeRgbStr, edgeRgbFrame)

        # key = cv2.waitKey(1)
        # if key == ord('q'):
        #     break

        # if key == ord('1'):
        #     print("Switching sobel filter kernel.")
        #     cfg = dai.EdgeDetectorConfig()
        #     sobelHorizontalKernel = [[1, 0, -1], [2, 0, -2], [1, 0, -1]]
        #     sobelVerticalKernel = [[1, 2, 1], [0, 0, 0], [-1, -2, -1]]
        #     cfg.setSobelFilterKernels(sobelHorizontalKernel, sobelVerticalKernel)
        #     edgeCfgQueue.send(cfg)

        # if key == ord('2'):
        #     print("Switching sobel filter kernel.")
        #     cfg = dai.EdgeDetectorConfig()
        #     sobelHorizontalKernel = [[3, 0, -3], [10, 0, -10], [3, 0, -3]]
        #     sobelVerticalKernel = [[3, 10, 3], [0, 0, 0], [-3, -10, -3]]
        #     cfg.setSobelFilterKernels(sobelHorizontalKernel, sobelVerticalKernel)
        #     edgeCfgQueue.send(cfg)

                    
        # if len(list1) == 0 or len(list2) ==0:
        #     # print("only 1 person detected in frame")
        #     continue
        # #   continue
        ##
        ##
        cv2.imshow('Movenet Multipose_cam 0', frame1)
        cv2.imshow('Movenet Multipose_cam 1', frame2)
        cv2.imshow('Movenet Multipose_cam 2', frame3)
        ##
        ##OAK RGB Camera stream
        ##
        # # Retrieve 'bgr' (opencv format) frame
        # cv2.imshow("rgb", inRgb.getCvFrame())

        # cv2.imshow("rgb", cv2.cvtColor(inRgb.getCvFrame(), cv2.COLOR_RGB2BGR))
            #cv2.imshow("left", inLeft.getCvFrame())
            #cv2.imshow("right", inRight.getCvFrame())
        
        ##
        ##OAK Stereo Vision  stream
        ##
              
        cv2.imshow("disparity", frame)
        # Available color maps: https://docs.opencv.org/3.4/d3/d50/group__imgproc__colormap.html
        frame = cv2.applyColorMap(frame, cv2.COLORMAP_JET)
        cv2.imshow("disparity_color", frame)
        # if cv2.waitKey(10) & 0xFF==ord('p'):
        #     time.pause()
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    cap1.release()
    cap2.release()
    cap3.release()

cv2.destroyAllWindows()


LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT_UPPERARM_1
LEFT_SHOULDER_BLADE_2touchedLEFT

In [6]:
frame.shape
frame1.shape

(480, 640, 3)